# `df.dx` refers to the entire mesh

**Author**: Weiwei Wang, Max Albert 

**Date**: Jan 2013

**Purpose**:

A piece of code to show that the assemble in dolfin 1.2 works as expected for meshes generated by Netgen.

Previously, there was a "bug" (or rather, very surprising behaviour) in that `df.dx` had the same meaning as `df.dx(0)`, i.e. it would only take cells in region 0 into account, not the whole mesh (see https://answers.launchpad.net/dolfin/+question/212697). Unfortunately, the markers in Netgen meshes start with 1, so this would lead to wrong results.

Fortunately, as of dolfin version 1.2 this is fixed and `df.dx` now refers to the whole mesh. This is illustrated in the following snippets.

In [1]:
import dolfin
import sys
from finmag.util.meshes import box

[2014-09-12 14:28:35] INFO: Finmag logging output will be appended to file: '/home/david/.finmag/global.log'
[2014-09-12 14:28:35] DEBUG: Building modules in 'native'...
[2014-09-12 14:28:36] DEBUG: FinMag          5430:23de5abc150f15c6eeda90e6bfc375599a63c112
[2014-09-12 14:28:36] DEBUG: Dolfin          1.4.0                Matplotlib      1.3.1               
[2014-09-12 14:28:36] DEBUG: Numpy           1.8.1                Scipy           0.14.0              
[2014-09-12 14:28:36] DEBUG: IPython         2.1.0                Python          2.7.6               
[2014-09-12 14:28:37] DEBUG: Paraview        4.0.1-1ubuntu1       Sundials        2.5.0               
[2014-09-12 14:28:37] DEBUG: Boost-Python    <unknown>            Linux           Ubuntu 14.04.1 LTS  
[2014-09-12 14:28:37] DEBUG: Registering debug signal handler. Press Ctrl-Z at any time to stop execution and jump into the debugger.


In [2]:
# IPYTHON_TEST_IGNORE_OUTPUT
print "sys.path: {}".format(sys.path)

sys.path: ['', '/home/david/hg/finmag/doc/ipython_notebooks_src', '/home/david/hg/finmag/src', '/usr/lib/python2.7', '/usr/lib/python2.7/plat-x86_64-linux-gnu', '/usr/lib/python2.7/lib-tk', '/usr/lib/python2.7/lib-old', '/usr/lib/python2.7/lib-dynload', '/usr/local/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages/PILcompat', '/usr/lib/python2.7/dist-packages/gtk-2.0', '/usr/lib/pymodules/python2.7', '/usr/lib/python2.7/dist-packages/ubuntu-sso-client', '/usr/lib/python2.7/dist-packages/wx-2.8-gtk2-unicode', '/usr/local/lib/python2.7/dist-packages/IPython/extensions']


In [3]:
import os
print "PYTHON_ROOTS: {}".format(os.environ.get('PYTHON_ROOTS'))

PYTHON_ROOTS: None


In [4]:
# IPYTHON_TEST_IGNORE_OUTPUT
print "Dolfin version:", dolfin.__version__

Dolfin version: 1.4.0


In [5]:
def test_assemble(mesh, descr):
    """
    Helper function which determines the volumes of all tetrahedra in the given mesh
    and checks that they are nonzero.

    """
    S1 = dolfin.FunctionSpace(mesh, "Lagrange", 1)
    volumes = dolfin.assemble(dolfin.TestFunction(S1) * dolfin.dx)
    all_nonzero = volumes.array().all()
    print("df.dx yields non-zero cell volumes on {} mesh: {}".format(descr, all_nonzero))
    assert(all_nonzero)

Now we check that both the mesh generated by dolfin and by Netgen lead to non-zero volumes when we apply `df.dx` on them.

In [8]:
mesh1 = dolfin.UnitCubeMesh(2, 2, 2)
test_assemble(mesh1, "dolfin")

df.dx yields non-zero cell volumes on dolfin mesh: True


In [9]:
mesh2 = box(0, 0, 0, 2, 2, 2, maxh=1.0)
test_assemble(mesh2, "netgen")

[2014-09-12 14:29:04] DEBUG: The mesh '/home/david/hg/finmag/doc/ipython_notebooks_src/box-0_0-0_0-0_0-2_0-2_0-2_0-1_0.xml.gz' already exists and is automatically returned.


df.dx yields non-zero cell volumes on netgen mesh: True
